In [1]:
import pandas as pd
import math
import numpy as np
from scipy.sparse import csr_matrix,lil_matrix


In [2]:

table = pd.read_excel('poem.xlsx')
word_list_df = pd.read_excel('wordlist.xlsx')

In [3]:
# 计算wordlist中词语的最大长度max_length
# TODO
n=word_list_df.shape[0]
max_length=0
for i in range(n):
    tmp_l=len(word_list_df.loc[i].word)
    if tmp_l>max_length:
        max_length=tmp_l
print(max_length)

3


In [4]:
word_list_set=set(word_list_df.word)

In [5]:
# 实现前向最大匹配分词算法
# string: 待分词的串
# word_list: 词表
# max_length: 词表中词语的最大长度
  
def cut(string, word_list, max_length):
    # TODO
    words = []
    segStr = string
    segStrLen = len(segStr)

    while segStrLen > 0:
        if segStrLen > max_length:
            wordLen = max_length
        else:
            wordLen = segStrLen
        subStr = segStr[0:wordLen]
        while wordLen > 1:
            if subStr in word_list:
                break
            else:
                wordLen = wordLen - 1
                subStr = subStr[0:wordLen]
        words.append(subStr)
        segStr = segStr[wordLen:]
        segStrLen = segStrLen - wordLen
#     for word in words:
#         print("word: ", word)
    return words


In [6]:
# 执行分词（可能需要等待几分钟）
table['words'] = table['content'].apply(lambda x: ' '.join(cut(x, word_list_set, max_length)))
# table

In [7]:
# 按照空格分开，stack
split_words = table['words'].str.split(' ', expand=True).stack().rename('word').reset_index()
new_data = pd.merge(table['Poem_id'], split_words, left_index=True, right_on='level_0')
# new_data.head(20)

In [8]:
poem_cnt=len(set(table.Poem_id))

In [9]:
cnts_all=new_data.groupby('word').count().Poem_id
cnts_all=cnts_all[33:20114]

In [10]:
word_list=[]
for s in cnts_all.index:
    word_list.append(s)##创建新的list

In [11]:
# word_list
# tf.sum(axis=1)

In [12]:
h=len(word_list)

In [13]:
groups=new_data.groupby(['Poem_id','word']).count()

In [14]:
poem_id_list=set()
for k1,k2 in groups.index:
    poem_id_list.add(k1)

In [15]:
poem_id_list=list(poem_id_list)
poem_id_list.sort()
w=len(poem_id_list)

In [16]:
tf=np.zeros((h,w))


In [17]:
word2pos=dict()
for i in range(h):
    word2pos[word_list[i]]=i
poem_id2pos=dict()
for i in range(w):
    poem_id2pos[poem_id_list[i]]=i

In [18]:
new_word_list=set(word_list)
for k1,k2 in groups.index:
    if k2 in new_word_list:
        n=groups.loc[(k1,k2)].level_0
        tf[word2pos[k2],poem_id2pos[k1]]=n
#         if n>1:print(k1,k2,n)

In [21]:
# np.save("word2pos.npy",word2pos)

In [23]:
# np.save("poem_id2pos.npy",poem_id2pos)

In [24]:
# np.save("tf_w2f.npy",tf)

In [19]:
# word2pos['阿']

In [19]:
cnt_more_10=[]
for i in range(h):
    if cnts_all[i]>10:
        cnt_more_10.append(i)

In [55]:
# len(cnt_more_10)

2762

In [25]:
# np.save("cnt_more_10.npy",cnt_more_10)

In [20]:
A=tf.copy()

In [21]:
A = np.where(A > 0, 1, 0)

In [22]:
tf_words=csr_matrix(A).dot(csr_matrix(tf.T))
tf_words=lil_matrix(tf_words)
tf_words.setdiag(0)
tf_words=csr_matrix(tf_words)
tf_words_dense=tf_words.todense()


In [23]:
# tf_words_dense[0].sum()

In [24]:

B=tf_words_dense.copy()
B1=np.where(B>0,1,0)



In [25]:
df_word=B1.sum(axis=0)

In [26]:
sum_of_word=len(word_list)
idf_word=[]
for i in df_word:
    idf_word.append(math.log(sum_of_word/i,2))

In [27]:
type(idf_word)

list

In [28]:
tf_idf_word=tf_words.multiply(csr_matrix(np.array(idf_word).reshape(-1,1)))

In [29]:
# tf_idf_word

<20081x20081 sparse matrix of type '<class 'numpy.float64'>'
	with 3225304 stored elements in Compressed Sparse Row format>

In [31]:
# from scipy import sparse
# sparse.save_npz('./tfidf_w2w.npz', tf_idf_word)

In [30]:
# tf_idf_word.toarray()

In [31]:
norm_tf_idf_word=tf_idf_word/np.sqrt(((tf_idf_word.multiply(tf_idf_word)).sum(axis=1)))

In [32]:
csr=csr_matrix(norm_tf_idf_word)
csrt=csr_matrix(norm_tf_idf_word.T)

In [33]:
sim=csr.dot(csrt)


In [34]:
sim_mat=sim[cnt_more_10]
sim_mat=sim_mat[:,cnt_more_10]

In [35]:
# sim_mat.toarray()

In [36]:
# sim_mat

In [37]:
# from scipy import sparse


# sparse.save_npz('./simmat.npz', sim_mat)  #保存

# csr_matrix_variable = sparse.load_npz('simmat.npz') #读

In [49]:
mat=sim_mat.toarray()

In [43]:
x_arr,y_arr=csr_matrix(sim_mat).nonzero()

In [44]:
len(x_arr)

7628644

In [74]:
# x_arr,y_arr=csr_matrix(sim_mat).nonzero()
new_res={}
line=0
tempdic={}
for k in range(len(x_arr)):
    x=x_arr[k]
    y=y_arr[k]
    while line!=x:
        new_res[word_list[cnt_more_10[line]]]=sorted(tempdic.items(), key=lambda x: x[1], reverse = True)
        tempdic={}
        line+=1
    if mat[x,y]>0.7:
        tempdic[word_list[cnt_more_10[y]]]=mat[x,y]
new_res[word_list[cnt_more_10[line]]]=sorted(tempdic.items(), key=lambda x: x[1], reverse = True)

In [78]:
# np.save('sim_char.npy', new_res)

In [3]:
test=np.load('sim_char.npy',allow_pickle=True).item()

In [6]:
# test['夜']

[('夜', 0.9999999999999643),
 ('盡', 0.8668940590349343),
 ('時', 0.8662877368918978),
 ('一', 0.8621513419338535),
 ('水', 0.8618186876373181),
 ('滿', 0.8593335241520279),
 ('在', 0.8576614533664821),
 ('風', 0.8575251199202591),
 ('向', 0.8558281413425892),
 ('秋', 0.8547383689927114),
 ('入', 0.8533960053887519),
 ('中', 0.8524443854322683),
 ('月', 0.8520027888439605),
 ('卻', 0.8504914404707239),
 ('下', 0.8502037894142019),
 ('到', 0.850167787741081),
 ('上', 0.8484074007882799),
 ('去', 0.8468873318668371),
 ('自', 0.8468689091139934),
 ('花', 0.8467150089586207),
 ('人', 0.8429092267488769),
 ('是', 0.8421192336745585),
 ('似', 0.8408174531449445),
 ('雲', 0.8403406394109729),
 ('來', 0.8402011068555129),
 ('過', 0.8385203958913439),
 ('樹', 0.8384552828065011),
 ('落', 0.837801830901409),
 ('聲', 0.836966938324714),
 ('更', 0.8359349114714061),
 ('得', 0.8354827928425751),
 ('新', 0.8353596381520325),
 ('日', 0.8350312267674215),
 ('空', 0.8347583440833211),
 ('看', 0.8343118440061391),
 ('猶', 0.83172922761838

In [38]:
# mat=csr_matrix_variable.toarray()
# x,y=np.where(mat>0.00001)
# len(x)

In [39]:
# sum_of_word

In [40]:
# sim_dense=sim.todense()
# sim_dense=sim_dense[cnt_more_10]
# sim_dense=sim_dense[:,cnt_more_10]